In [13]:
import re
import sys
import os
# !pip3 install nest_asyncio
# !pip3 install twint
# !pip3 install nltk
#pip install tweet-preprocessor
import nest_asyncio
import string
from datetime import datetime
nest_asyncio.apply()
import twint
import pandas as pd
import nltk
import pytz
import textblob
import json
import itertools
from elasticsearch import Elasticsearch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer, WhitespaceTokenizer
import preprocessor as p

#CORPORA
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaMulticore
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel

#PYLDAVIS
import pyLDAvis
import pyLDAvis.gensim

#p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)
nltk.download('stopwords')
#%run "candidates.ipynb"

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dominic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:

class TwitterClient:
    '''
    Twitter Twint intialization
    '''
    def __init__(self):
        #Intializes the Twint object
        self.twint_Api = twint.Config()

    def preprocess(self, tweets):
        for i in range(len(tweets)):
            tweets.at[i, 'tweet'] = p.clean( tweets.at[i, 'tweet']) 
        return tweets
    
    def clean_tweets(self, data ):
        cleaned_tweets = {}
        newdata = []

        stop_words = stopwords.words('english')
        words = set(nltk.corpus.words.words())

        for token in data.tweet: 
            newdoc = ""
            #print(token)
            seperator = " " 
            token = re.sub('[^A-Za-z0-9]+', ' ', token) #remove special characters
            token = re.sub(r'\d+', '', token) #remove numbers
            token = re.sub(r'\b\w{1,2}\b', '', token) #remove words with <= 2 characters
            token = " ".join(w for w in nltk.wordpunct_tokenize(token) \
                    # if w.lower() in words or 
                     or not w.isalpha())
            whitespace_token = WhitespaceTokenizer().tokenize( token )
            wo_stopwords_token = [x for x in whitespace_token
                                      if not x in stop_words]
            newdoc = seperator.join( (wo_stopwords_token) ).lower()

            #create word tokens for each tweet
            word_tokens = RegexpTokenizer('\s+', gaps=True).tokenize(newdoc)

            #data[idx] = newdoc
            newdata.append( word_tokens )

        return newdata

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(tweet)
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity

    def twint_to_pandas(self, col):
        return twint.output.panda.Tweets_df[col]

    def get_tweets(self, count):

        #Creates a search string from the list of presidential candidates
        self.twint_Api.Username = '@realDonaldTrump'
        self.twint_Api.Limit = count
        self.twint_Api.Format = "Date: {date} | Tweet: {tweet}"
        self.twint_Api.Location = True
        self.twint_Api.Lang = "en"
        self.twint_Api.Pandas = True
        #self.Output = "output.csv"
        #self.twint_Api.Popular_tweets = True
        #self.twint_Api.Pandas_au = True
        #self.twint_Api.Timedelta = 340
        tweets_df = pd.DataFrame(columns=["date", "tweet"])
        self.twint_Api.Hide_output = True
        
        twint.run.Search(self.twint_Api)
        
        tweets_df = tweets_df.append(self.twint_to_pandas(["date", "tweet"]), ignore_index=True)
        
        
        
        
        tweets_df = self.preprocess(tweets_df)
        #print(tweets_df)
        toks = tweets_df = self.clean_tweets(tweets_df)
        #tweets_df
        
        return tweets_df

<>:37: DeprecationWarning: invalid escape sequence \s
<>:37: DeprecationWarning: invalid escape sequence \s
<>:37: DeprecationWarning: invalid escape sequence \s
<ipython-input-14-1d660f054867>:37: DeprecationWarning: invalid escape sequence \s
  word_tokens = RegexpTokenizer('\s+', gaps=True).tokenize(newdoc)


In [15]:
 def bow_lda_model(data):
    l_dictionary = Dictionary(data)
    print("Dictionary Loaded")
    bow_corpus = [l_dictionary.doc2bow(doc) for doc in data]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=l_dictionary,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)
        
    vocab = list(l_dictionary.values()) #list of terms in the dictionary
    
    vocab_tf = [dict(i) for i in bow_corpus]
    vocab_tf = list(pd.DataFrame(vocab_tf).sum(axis=0)) #list of term frequencies
    
    #bow = pd.DataFrame(vocab_tf, index=vocab, columns=["term_freq"])
    
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary=l_dictionary)

    pyLDAvis.save_html(vis, 'lda.html')
    
    bow = dict(zip(vocab, vocab_tf))
    #bow.to_dict('index')
   # print( bow.keys() )
        
    top_terms = []
    for j in range(5):
        c_terms = []
        for i in range(len(lda_model.show_topic(j))):
            c_terms.append(lda_model.show_topic(j)[i][0] )
        top_terms.append(c_terms)
        
    print( top_terms )
    
    final_list = {
                    "name": "Donald Trump",
                    "children": []
    }
    
    for l in range(len(top_terms)):
        c_topic = { 
                        "name": str(l),
                        "children": []
                      }
        for m in range(len(top_terms[l])):
            c_topic["children"].append({ "name": top_terms[l][m],
                                         "size": bow[ top_terms[l][m] ] })
            
        final_list["children"].append( c_topic )
    
    #print( final_list )
    
    with open('json/trump.json', 'w') as json_file:
        json.dump(final_list, json_file)


In [16]:
d = TwitterClient()
tweets = d.get_tweets(1000)

bow_lda_model( tweets )


Dictionary Loaded


/home/dominic/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


[['nothing', 'get', 'they', 'country', 'states', 'united', 'said', 'ukraine', 'time', 'ukrainian'], ['republican', 'election', 'thank', 'state', 'must', 'party', 'usa', 'pro', 'john', 'biden'], ['schiff', 'news', 'whistleblower', 'even', 'congress', 'got', 'adam', 'fake', 'hunt', 'media'], ['president', 'statement', 'turkey', 'good', 'all', 'would', 'making', 'tough', 'isis', 'fighting'], ['the', 'great', 'democrats', 'this', 'trump', 'impeachment', 'many', 'border', 'going', 'job']]
